In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

# Load and preprocess the data
def load_data():
    print("Loading data...")
    X_ps1 = pd.read_csv(r"C:\Users\HP\Downloads\Mahak_btp\VSCode_FL_Mahak\FL_Valve_Classification\PS1.csv", header=None).values
    X_ps2 = pd.read_csv(r"C:\Users\HP\Downloads\Mahak_btp\VSCode_FL_Mahak\FL_Valve_Classification\PS2.csv", header=None).values
    y = pd.read_csv(r"C:\Users\HP\Downloads\Mahak_btp\VSCode_FL_Mahak\FL_Valve_Classification\valve_target_encoded.csv", header=None).values.flatten()
    print(f"Data shapes: X_ps1={X_ps1.shape}, X_ps2={X_ps2.shape}, y={y.shape}")

    # Combine features from PS1 and PS2
    X_combined = np.concatenate([X_ps1, X_ps2], axis=1)
    print(f"Combined data shape: {X_combined.shape}")

    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_combined)
    print("Data normalization complete.")

    # Reshape to (samples, timesteps, features) for LSTM input
    X_reshaped = X_scaled.reshape(2205, 1, 12000)
    print(f"Reshaped data for LSTM: {X_reshaped.shape}")

    return X_reshaped, y


# Define the LSTM model structure
def create_model(input_shape):
    print("Creating LSTM model...")
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.LSTM(256, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(128, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(64, return_sequences=False),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(4, activation="softmax")  # For 4 classes
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    print("Model created successfully.")
    return model


# Simulate a federated learning client
class FederatedClient:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.model = create_model(input_shape=(1, 12000))

    def train(self, X_train, y_train):
        print("Training client model...")
        self.model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)
        print("Client model training complete.")
        return self.model.get_weights()

    def evaluate(self, X_eval, y_eval, set_name="Test"):
        print(f"Evaluating client model on {set_name} data...")
        loss, accuracy = self.model.evaluate(X_eval, y_eval, verbose=0)
        print(f"Client model {set_name} data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return loss, accuracy


# Main federated learning simulation
def federated_learning_simulation(num_clients):
    # Load data
    X_total, y_total = load_data()

    # Split the data for train and test, stratified by class labels
    print("Splitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_total, y_total, test_size=0.10, random_state=104, stratify=y_total)
    print(f"Training data shape: {X_train.shape}, Testing data shape: {X_test.shape}")

    # Check class distribution in train and test sets
    print(f"Class distribution in training set: {Counter(y_train)}")
    print(f"Class distribution in testing set: {Counter(y_test)}")

    # Split data across clients, ensuring fairness
    print(f"Simulating {num_clients} federated clients...")
    data_per_client = len(X_train) // num_clients
    clients = []
    for i in range(num_clients):
        X_client = X_train[i * data_per_client:(i + 1) * data_per_client]
        y_client = y_train[i * data_per_client:(i + 1) * data_per_client]

        # Split each client data into training and testing sets
        X_client_train, X_client_test, y_client_train, y_client_test = train_test_split(
            X_client, y_client, test_size=0.1, random_state=42, stratify=y_client)

        # Verify client-level class distribution
        print(f"Client {i + 1} - Class distribution: {Counter(y_client_train)} (Train), {Counter(y_client_test)} (Test)")

        client = FederatedClient(X_client, y_client)
        clients.append((client, X_client_train, y_client_train, X_client_test, y_client_test))
        print(f"Client {i + 1} - Train data shape: {X_client_train.shape}, Test data shape: {X_client_test.shape}")

    # Train each client model and aggregate weights
    global_weights = None
    print("Training and aggregating client models...")
    for i, (client, X_client_train, y_client_train, X_client_test, y_client_test) in enumerate(clients):
        print(f"Training client {i + 1}...")
        client_weights = client.train(X_client_train, y_client_train)
        
        # Evaluate client on its own training and testing data
        train_loss, train_accuracy = client.evaluate(X_client_train, y_client_train, set_name="Training")
        test_loss, test_accuracy = client.evaluate(X_client_test, y_client_test, set_name="Testing")

        # Aggregate client weights without training the global model
        if global_weights is None:
            global_weights = client_weights
        else:
            # Average weights for federated aggregation
            global_weights = [(g + c) / 2 for g, c in zip(global_weights, client_weights)]
    
    print("Client model training and aggregation complete.")

    # Print accuracies for each client
    for i, (client, X_client_train, y_client_train, X_client_test, y_client_test) in enumerate(clients):
        print(f"Client {i + 1} - Final Evaluation on Test Data:")
        client.evaluate(X_client_test, y_client_test, set_name="Final Test")

    # Set global model with aggregated weights (but no global training)
    global_model = create_model(input_shape=(1, 12000))
    global_model.set_weights(global_weights)

    # Evaluate global model on the original test data (using aggregated weights)
    print("Evaluating global model on overall test data...")
    test_loss, test_accuracy = global_model.evaluate(X_test, y_test, verbose=0)
    print(f"Global model on overall test data - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")


if __name__ == "__main__":
    # Number of simulated clients
    NUM_CLIENTS = 3
    federated_learning_simulation(NUM_CLIENTS)


Loading data...
Data shapes: X_ps1=(2205, 6000), X_ps2=(2205, 6000), y=(2205,)
Combined data shape: (2205, 12000)
Data normalization complete.
Reshaped data for LSTM: (2205, 1, 12000)
Splitting data into train and test sets...
Training data shape: (1984, 1, 12000), Testing data shape: (221, 1, 12000)
Class distribution in training set: Counter({0: 1012, 2: 324, 1: 324, 3: 324})
Class distribution in testing set: Counter({0: 113, 3: 36, 2: 36, 1: 36})
Simulating 3 federated clients...
Client 1 - Class distribution: Counter({0: 315, 1: 97, 3: 95, 2: 87}) (Train), Counter({0: 35, 3: 11, 1: 11, 2: 10}) (Test)
Creating LSTM model...
Model created successfully.
Client 1 - Train data shape: (594, 1, 12000), Test data shape: (67, 1, 12000)
Client 2 - Class distribution: Counter({0: 297, 2: 101, 3: 99, 1: 97}) (Train), Counter({0: 33, 2: 12, 1: 11, 3: 11}) (Test)
Creating LSTM model...
Model created successfully.
Client 2 - Train data shape: (594, 1, 12000), Test data shape: (67, 1, 12000)
Clie

In [ ]:
#0.8643


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

# Load and preprocess the data
def load_data():
    print("Loading data...")
    X_ps1 = pd.read_csv(r"C:\Users\SystemDynamicsLab\OneDrive - IIT Indore\Desktop\Mahak_BTP_Folder\Mahak\Mahak_btp\Dataset_hydraulic-20240917T065446Z-001\Dataset_hydraulic\FL_valve_classification_PS1_PS2\PS1.csv", header=None).values
    X_ps2 = pd.read_csv(r"C:\Users\SystemDynamicsLab\OneDrive - IIT Indore\Desktop\Mahak_BTP_Folder\Mahak\Mahak_btp\Dataset_hydraulic-20240917T065446Z-001\Dataset_hydraulic\FL_valve_classification_PS1_PS2\PS2.csv", header=None).values
    y = pd.read_csv(r"C:\Users\SystemDynamicsLab\OneDrive - IIT Indore\Desktop\Mahak_BTP_Folder\Mahak\Mahak_btp\Dataset_hydraulic-20240917T065446Z-001\Dataset_hydraulic\FL_valve_classification_PS1_PS2\valve_target_encoded.csv", header=None).values.flatten()
    print(f"Data shapes: X_ps1={X_ps1.shape}, X_ps2={X_ps2.shape}, y={y.shape}")

    # Combine features from PS1 and PS2
    X_combined = np.concatenate([X_ps1, X_ps2], axis=1)
    print(f"Combined data shape: {X_combined.shape}")

    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_combined)
    print("Data normalization complete.")

    # Reshape to (samples, timesteps, features) for LSTM input
    X_reshaped = X_scaled.reshape(2205, 1, 12000)
    print(f"Reshaped data for LSTM: {X_reshaped.shape}")

    return X_reshaped, y


# Define the LSTM model structure
def create_model(input_shape):
    print("Creating LSTM model...")
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.LSTM(256, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(128, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(64, return_sequences=False),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(4, activation="softmax")  # For 4 classes
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    print("Model created successfully.")
    return model


# Simulate a federated learning client
class FederatedClient:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.model = create_model(input_shape=(1, 12000))

    def train(self, X_train, y_train):
        print("Training client model...")
        self.model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)
        print("Client model training complete.")
        return self.model.get_weights()

    def evaluate(self, X_eval, y_eval, set_name="Test"):
        print(f"Evaluating client model on {set_name} data...")
        loss, accuracy = self.model.evaluate(X_eval, y_eval, verbose=0)
        print(f"Client model {set_name} data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return loss, accuracy


# Main federated learning simulation
def federated_learning_simulation(num_clients):
    # Load data
    X_total, y_total = load_data()

    # Split the data for train and test, stratified by class labels
    print("Splitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_total, y_total, test_size=0.10, random_state=104, stratify=y_total)
    print(f"Training data shape: {X_train.shape}, Testing data shape: {X_test.shape}")

    # Check class distribution in train and test sets
    print(f"Class distribution in training set: {Counter(y_train)}")
    print(f"Class distribution in testing set: {Counter(y_test)}")

    # Split data across clients, ensuring fairness
    print(f"Simulating {num_clients} federated clients...")
    data_per_client = len(X_train) // num_clients
    clients = []
    for i in range(num_clients):
        X_client = X_train[i * data_per_client:(i + 1) * data_per_client]
        y_client = y_train[i * data_per_client:(i + 1) * data_per_client]

        # Split each client data into training and testing sets
        X_client_train, X_client_test, y_client_train, y_client_test = train_test_split(
            X_client, y_client, test_size=0.1, random_state=42, stratify=y_client)

        # Verify client-level class distribution
        print(f"Client {i + 1} - Class distribution: {Counter(y_client_train)} (Train), {Counter(y_client_test)} (Test)")

        client = FederatedClient(X_client, y_client)
        clients.append((client, X_client_train, y_client_train, X_client_test, y_client_test))
        print(f"Client {i + 1} - Train data shape: {X_client_train.shape}, Test data shape: {X_client_test.shape}")

    # Train each client model and aggregate weights
    global_weights = None
    print("Training and aggregating client models...")
    for i, (client, X_client_train, y_client_train, X_client_test, y_client_test) in enumerate(clients):
        print(f"Training client {i + 1}...")
        client_weights = client.train(X_client_train, y_client_train)
        
        # Evaluate client on its own training and testing data
        train_loss, train_accuracy = client.evaluate(X_client_train, y_client_train, set_name="Training")
        test_loss, test_accuracy = client.evaluate(X_client_test, y_client_test, set_name="Testing")

        # Aggregate client weights without training the global model
        if global_weights is None:
            global_weights = client_weights
        else:
            # Average weights for federated aggregation
            global_weights = [(g + c) / 2 for g, c in zip(global_weights, client_weights)]
    
    print("Client model training and aggregation complete.")

    # Print accuracies for each client
    for i, (client, X_client_train, y_client_train, X_client_test, y_client_test) in enumerate(clients):
        print(f"Client {i + 1} - Final Evaluation on Test Data:")
        client.evaluate(X_client_test, y_client_test, set_name="Final Test")

    # Set global model with aggregated weights (but no global training)
    global_model = create_model(input_shape=(1, 12000))
    global_model.set_weights(global_weights)

    # Evaluate global model on the original test data (using aggregated weights)
    print("Evaluating global model on overall test data...")
    test_loss, test_accuracy = global_model.evaluate(X_test, y_test, verbose=0)
    print(f"Global model on overall test data - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")


if __name__ == "__main__":
    # Number of simulated clients
    NUM_CLIENTS = 3
    federated_learning_simulation(NUM_CLIENTS)


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [3]:
pip show tensorflow


Name: tensorflow
Version: 2.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\systemdynamicslab\appdata\roaming\python\python39\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade tensorflow


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 390.0/390.0 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 15.9/15.9 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 126.7/126.7 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 4.0 MB/s eta 0:00:00
     -------------------------------------- 242.4/242.4 kB 4.9 MB/s eta 0:00:00
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\SystemDynamicsLab\\AppData\\Roaming\\Python\\Python39\\site-packages\\~~mpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Check the permissions.



In [5]:
pip install --upgrade numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
